In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from boruta import BorutaPy

from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go

In [2]:
lagdf = pd.read_csv("df_all_lag.csv")
lagdf.drop(['Unnamed: 0'], axis=1, inplace=True)
lagdf

,OIL_D(t-4),OIL_B(t-4),BEAN(t-4),GOLD(t-4),NIKEL(t-4),ZINC(t-4),CORN(t-4),WHEAT(t-4),COPP(t-4),ALMN(t-4),...,Employ(t=0),CLI_OECD(t=0),Sprate(t=0),EFFR_US(t=0),KRWUSD(t=0),KRWJPY(t=0),KRWEUR(t=0),JPYUSD(t=0),USDEUR(t=0),ln_amount(t=0)
0,-1.342275,-1.309657,-1.359974,-1.472582,-0.943449,-1.126503,-1.091638,-1.429193,-1.575211,-0.529093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.296990,-1.238007,-1.379677,-1.455909,-0.805464,-1.184781,-1.121899,-1.478494,-1.629184,-0.877474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.350851,-1.404290,-1.250515,-1.481053,-0.760883,-1.142001,-0.994800,-1.397215,-1.614281,-0.978263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.382757,-1.427610,-1.305974,-1.491651,-0.772446,-1.078325,-1.068941,-1.503810,-1.617100,-1.116301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.289442,-1.238345,-1.332245,-1.494430,-0.898353,-1.126213,-1.061376,-1.330592,-1.610253,-1.151358,...,-1.926718,0.633035,-0.309988,1.256379,-0.056832,0.023550,-1.273599,-0.196298,-1.158479,17.024621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,0.926429,0.692164,1.140805,1.166659,0.633347,1.074024,1.677318,2.116453,0.781874,0.400471,...,1.736347,0.881747,-0.421964,-0.479027,1.288336,-0.503399,-0.067939,1.549865,-0.886104,18.641900
273,0.924371,0.932464,1.264129,1.114799,0.722381,0.750417,1.762051,1.907259,0.717026,0.536318,...,1.717978,0.619501,-0.277624,-0.248867,1.360726,-0.705525,-0.027672,1.949838,-0.892520,18.584752
274,0.587135,0.706021,1.446562,1.373468,1.388790,1.115817,1.583507,1.316984,1.000988,1.041909,...,1.728900,0.323069,-0.114919,0.002122,1.651932,-0.678358,-0.126643,2.134845,-1.122938,18.647829
275,0.533960,0.652283,1.591778,1.479814,1.773450,1.061254,1.683371,1.426245,1.056572,0.827205,...,1.742305,0.047201,-0.407316,0.364893,1.758918,-0.564481,-0.116653,2.019247,-1.168292,18.624712


In [3]:
lag_ln0 = lagdf['ln_amount(t=0)']
lag_ln0 = lag_ln0.drop(index=[0,1,2,3])
lag_ln0

4      17.024621
5      17.022301
6      17.170375
7      17.106393
8      17.144858
         ...    
272    18.641900
273    18.584752
274    18.647829
275    18.624712
276    18.587797
Name: ln_amount(t=0), Length: 273, dtype: float64

In [4]:
lagT4 = lagdf.iloc[:, 0:48]
lagT4 = pd.concat([lagT4, lag_ln0], axis=1)

In [5]:
def split_lagdf(df):
    return df[:194], df[194:]

In [6]:
df_train, df_test = split_lagdf(lagT4)

In [7]:
df_train.drop(df_train.index[0:4], inplace=True)
df_test.drop(df_test.index[0:4], inplace=True)

C:\Users\koojy\AppData\Local\Temp\ipykernel_18416\3515249492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(df_train.index[0:4], inplace=True)
C:\Users\koojy\AppData\Local\Temp\ipykernel_18416\3515249492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(df_test.index[0:4], inplace=True)


In [8]:
def xy_tt(df_train, df_test):    
    X_train = df_train
    y_train = df_train['ln_amount(t=0)']
    X_test = df_test
    y_test= df_test['ln_amount(t=0)']
    
    return X_train, y_train, X_test, y_test

In [9]:
X_train, y_train, X_test, y_test = xy_tt(df_train, df_test)

In [10]:
X_train.drop(['ln_amount(t=0)'], axis=1, inplace=True)
X_test.drop(['ln_amount(t=0)'], axis=1, inplace=True)

C:\Users\koojy\AppData\Local\Temp\ipykernel_18416\1366950026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['ln_amount(t=0)'], axis=1, inplace=True)
C:\Users\koojy\AppData\Local\Temp\ipykernel_18416\1366950026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(['ln_amount(t=0)'], axis=1, inplace=True)


In [11]:
model = RandomForestRegressor(n_estimators='auto', max_depth=5, random_state=42)

# initialize Boruta
feat_selector = BorutaPy(verbose=2, estimator=model, n_estimators='auto',
                        max_iter=10)

feat_selector.fit(np.array(X_train), np.array(y_train))

print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        print("Passes the test: ", X_train.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              X_train.columns[i], " - Ranking: ", feat_selector.ranking_[i])

Iteration: 	1 / 10
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	2 / 10
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	3 / 10
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	4 / 10
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	5 / 10
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	6 / 10
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	7 / 10
Confirmed: 	0
Tentative: 	48
Rejected: 	0
Iteration: 	8 / 10
Confirmed: 	24
Tentative: 	13
Rejected: 	11
Iteration: 	9 / 10
Confirmed: 	24
Tentative: 	13
Rejected: 	11


BorutaPy finished running.

Iteration: 	10 / 10
Confirmed: 	24
Tentative: 	4
Rejected: 	11

------Support and Ranking for each feature------
Passes the test:  OIL_D(t-4)  - Ranking:  1
Passes the test:  OIL_B(t-4)  - Ranking:  1
Doesn't pass the test:  BEAN(t-4)  - Ranking:  13
Passes the test:  GOLD(t-4)  - Ranking:  1
Passes the test:  NIKEL(t-4)  - Ranking:  1
Doesn't pass the test:  ZINC(t-4)  - Ranking:  2
Doesn't pass the test:  CORN(t

In [24]:
accept = X_train.columns[feat_selector.support_].to_list()
X_test = X_test[accept]

In [25]:
ln_XG = XGBRegressor()

ln_XG.fit(X_test, y_test)

predictions_XG = ln_XG.predict(X_test)

df_boruta = pd.DataFrame({'pred': predictions_XG, 'observed': y_test})
df_boruta

,pred,observed
198,18.150417,18.150798
199,18.112312,18.112392
200,18.103426,18.102666
201,18.185568,18.186215
202,18.122990,18.122995
...,...,...
272,18.642084,18.641900
273,18.585094,18.584752
274,18.647482,18.647829
275,18.624588,18.624712


In [26]:
df_est = lag_ln0.loc[:194] #train만큼 사용한 부분 
ml_pred = pd.DataFrame(predictions_XG)    # XG Boost 사용한 predictions
df_est_1 = pd.concat([df_est,ml_pred], axis=0, ignore_index=True) #train과 test합치기
df_est_1.columns = ['estimation']

In [27]:
R2 = r2_score(df_boruta['pred'], df_boruta['observed'])
print("r^2: ", R2*100)

MAE = np.abs(np.subtract(df_boruta['pred'], df_boruta['observed'])).mean()
print('MAE: ', MAE)

r^2:  99.99837136866904
MAE:  0.0004629086063154285


In [28]:
fig = px.line(df_est_1, x=df_est_1.index, y='estimation' ,title='ROK EX_IM')
fig.add_trace(go.Scatter(x=lag_ln0.index, y=lag_ln0, mode='lines', name='Original'))

fig.show()

In [29]:
accept

['OIL_D(t-4)',
 'OIL_B(t-4)',
 'GOLD(t-4)',
 'NIKEL(t-4)',
 'WHEAT(t-4)',
 'COPP(t-4)',
 'ALMN(t-4)',
 'COTT(t-4)',
 'STI(t-4)',
 'SHCOMP(t-4)',
 'OIL_W(t-4)',
 'PPI_US(t-4)',
 'PPI_KR(t-4)',
 'IIP_JP(t-4)',
 'IIP_RU(t-4)',
 'IIP(t-4)',
 'SHIP_IDX(t-4)',
 'STOCK_IDX(t-4)',
 'CLI(t-4)',
 'CCI(t-4)',
 'LCI(t-4)',
 'Employ(t-4)',
 'Sprate(t-4)',
 'EFFR_US(t-4)']

In [39]:
dapr = pd.read_csv("ttApril.csv", thousands=',')
dapr

,OIL_D(t-4),OIL_B(t-4),GOLD(t-4),NIKEL(t-4),WHEAT(t-4),COPP(t-4),ALMN(t-4),COTT(t-4),STI(t-4),SHCOMP(t-4),...,IIP_RU(t-4),IIP(t-4),SHIP_IDX(t-4),STOCK_IDX(t-4),CLI(t-4),CCI(t-4),LCI(t-4),Employ(t-4),Sprate(t-4),EFFR_US(t-4)
0,78.89,81.31,1990.0,24211,619.75,8577,2368.0,79.03,3270.51,3323.28,...,NaN,109.5,98.2,124.5,109.4,110,113.4,28432,NaN,4.83


In [42]:
preds = ln_XG.predict(dapr)
preds

array([18.490917], dtype=float32)